In [1]:
!pip install requests BeautifulSoup4

import urllib.request, json, re
import requests
from bs4 import BeautifulSoup
from random import shuffle
from tqdm import tqdm

In [2]:
!pip install --upgrade firebase-admin

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account
cred = credentials.Certificate('tagger-12345-firebase-adminsdk-abcde-111aaa22bbb.json')
firebase_admin.initialize_app(cred)

db = firestore.client()

Requirement already up-to-date: firebase-admin in /usr/local/lib/python3.7/site-packages (3.1.0)


In [3]:
# Use this only to clear an existing DB (deletes everything!)

# def delete_collection(coll_ref, batch_size):
#     docs = coll_ref.limit(batch_size).get()
#     deleted = 0

#     for doc in docs:
#         doc.reference.delete()
#         deleted = deleted + 1

#     if deleted >= batch_size:
#         return delete_collection(coll_ref, batch_size)

# for col in ["articles", "users", "meta"]:
#     delete_collection(db.collection(col), 200)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 'Query.get' is deprecated:  please use 'Query.stream' instead.
  


In [4]:
base_url = "https://docs.microsoft.com/de-de/azure/azure-functions/"
hrefs = []

def recurse(node):
    if "children" in node:
        for child in node["children"]:
            recurse(child)
    elif not node["href"].startswith(".") and not node["href"].startswith("http"):
        hrefs.append(node["href"])

with urllib.request.urlopen(base_url + "toc.json") as url:
    data = json.loads(url.read().decode())
    for cat in data["items"]:
        recurse(cat)

shuffle(hrefs)
len(hrefs), hrefs[:3]

(135,
 ['functions-continuous-deployment',
  'functions-bindings-cosmosdb',
  'functions-integrate-storage-queue-output-binding'])

In [5]:
def get_docs(url):
    docs = []

    r = requests.get(url)
    html = r.text.encode(r.encoding)
    soup = BeautifulSoup(html, "html.parser")

    main = soup.find("main")
    if not main:
        return []
    paragraphs = main.find_all("p")

    doc = ""
    length = 0

    for p in paragraphs:
        if not p.find("span") or p.find("code") or p.find("a"):
            continue
        subtext = p.find("span").text.strip()
        added_len = len(subtext.split(" "))
        if added_len < 8:
            continue
        if length + added_len > 100:
            docs.append(doc.strip())
            doc = ""
            length = 0
        else:
            doc += subtext + " "
            length += added_len

    if length > 100:
        docs.append(doc.strip())

    return docs

get_docs(base_url + hrefs[5])

['Wenn Sie nur die Hauptversion („~2“ für 2.x oder „~1“ für 1.x) angeben, wird die Funktions-App automatisch auf neue Nebenversionen der Runtime aktualisiert, sobald diese verfügbar werden. Eine Änderung an der Runtimeversion bewirkt, dass eine Funktionen-App neu gestartet wird. Gehen Sie folgendermaßen vor, um die zurzeit von einer Funktions-App verwendete Runtimeversion anzuzeigen und zu aktualisieren. Wählen Sie unter Konfigurierte Features die Option Funktionen-App-Einstellungen aus. Um Ihre Funktions-App an die Runtime der Version 1.x anzuheften, wählen Sie ~1 unter Laufzeitversion aus.']

In [6]:
complete = []

for href in tqdm(hrefs):
    doc_url = base_url + href
    docs = get_docs(doc_url)
    
    for doc in docs:
        complete.append((doc_url, doc))
    
len(complete)

100%|██████████| 135/135 [01:01<00:00,  2.01it/s]


235

In [7]:
def process(doc):
    split = re.split(r"([^A-Za-zÀ-ž]+)", doc)
    allWords = []
    for s in split:
        base = re.sub(r"[^A-Za-zÀ-ž]+", "", s)
        if s:
            allWords.append({
                "complete": s,
                "word": base,
            })
    words = []
    skipNext = False
    for i, word in enumerate(allWords):
        if skipNext:
            skipNext = False
            continue
        followedByBlank = i < len(allWords) -1 \
            and allWords[i + 1]["complete"] == " "
        if followedByBlank:
            skipNext = True
        words.append({
            "complete": word["complete"],
            "word": word["word"],
            "addBlank": followedByBlank,
        })
    return words

process(complete[0][1])[:5]

[{'complete': 'Continuous', 'word': 'Continuous', 'addBlank': True},
 {'complete': 'Deployment', 'word': 'Deployment', 'addBlank': True},
 {'complete': 'eignet', 'word': 'eignet', 'addBlank': True},
 {'complete': 'sich', 'word': 'sich', 'addBlank': True},
 {'complete': 'hervorragend', 'word': 'hervorragend', 'addBlank': True}]

In [8]:
totalWordCount = 0
totalDocCount = 0

for url, doc in tqdm(complete):
    words = process(doc)
    
    if totalWordCount > 10000:
        print("Stopping prematurely")
        break

    db.collection(u'articles').add({
        u'original': doc,
        u'url': url,
        u'words': words,
        u'annotatorCount': 0,
        u'annotators': []
    })
    totalWordCount += len(words)
    totalDocCount += 1

db.collection(u'meta').document(u'information').set({
    u'documents': totalDocCount,
    u'words': totalWordCount,
})

print("Upload complete!")
print(totalDocCount, "documents")
print(totalWordCount, "words")

 37%|███▋      | 88/235 [00:21<00:31,  4.71it/s]

Stopping prematurely
Upload complete!
88 documents
10078 words
